In [3]:
import numpy as np

def rot(m,x,y,z,vx,vy,vz):

    jx=sum(m*(y*vz-z*vy))
    jy=sum(m*(z*vx-x*vz))
    jz=sum(m*(x*vy-y*vx))

    j=np.sqrt(jx*jx+jy*jy+jz*jz)

    rjx=jx/j
    rjy=jy/j
    rjz=jz/j

    e1x=rjy*rjz
    e1y=-rjx*rjz
    e1z=0.
    e1=np.sqrt(e1x**2+e1y**2+e1z**2)
    e1x=e1x/e1
    e1y=e1y/e1
    e1z=e1z/e1

    e2x=rjx*rjz**2
    e2y=rjy*rjz**2
    e2z=-(rjx**2+rjy**2)*rjz
    e2=np.sqrt(e2x**2+e2y**2+e2z**2)
    e2x=e2x/e2
    e2y=e2y/e2
    e2z=e2z/e2

    e3x=rjx
    e3y=rjy
    e3z=rjz
    e3=np.sqrt(e3x**2+e3y**2+e3z**2)
    e3x=e3x/e3
    e3y=e3y/e3
    e3z=e3z/e3

    xn=e1x*x+e1y*y+e1z*z
    yn=e2x*x+e2y*y+e2z*z
    zn=e3x*x+e3y*y+e3z*z
    vxn=e1x*vx+e1y*vy+e1z*vz
    vyn=e2x*vx+e2y*vy+e2z*vz
    vzn=e3x*vx+e3y*vy+e3z*vz
        
    return e1x,e2x,e3x,e1y,e2y,e3y,e1z,e2z,e3z 

def rot1(m,x,y,z,vx,vy,vz,rlim=30.):

    r=np.sqrt(x**2+y**2+z**2)
    kk,=np.where(r<rlim)
    jx=sum(m[kk]*(y[kk]*vz[kk]-z[kk]*vy[kk]))
    jy=sum(m[kk]*(z[kk]*vx[kk]-x[kk]*vz[kk]))
    jz=sum(m[kk]*(x[kk]*vy[kk]-y[kk]*vx[kk]))

    j=np.sqrt(jx*jx+jy*jy+jz*jz)

    rjx=jx/j
    rjy=jy/j
    rjz=jz/j

    e1x=rjy*rjz
    e1y=-rjx*rjz
    e1z=0.
    e1=np.sqrt(e1x**2+e1y**2+e1z**2)
    e1x=e1x/e1
    e1y=e1y/e1
    e1z=e1z/e1

    e2x=rjx*rjz**2
    e2y=rjy*rjz**2
    e2z=-(rjx**2+rjy**2)*rjz
    e2=np.sqrt(e2x**2+e2y**2+e2z**2)
    e2x=e2x/e2
    e2y=e2y/e2
    e2z=e2z/e2

    e3x=rjx
    e3y=rjy
    e3z=rjz
    e3=np.sqrt(e3x**2+e3y**2+e3z**2)
    e3x=e3x/e3
    e3y=e3y/e3
    e3z=e3z/e3

    xn=e1x*x+e1y*y+e1z*z
    yn=e2x*x+e2y*y+e2z*z
    zn=e3x*x+e3y*y+e3z*z
    vxn=e1x*vx+e1y*vy+e1z*vz
    vyn=e2x*vx+e2y*vy+e2z*vz
    vzn=e3x*vx+e3y*vy+e3z*vz
        
    return xn,yn,zn,vxn,vyn,vzn 


In [8]:
"""Class object that calculate bar strength profiles"""

import numpy as np
import bines
import math


class properties:

        def __init__(self,m,pos,vel=None,
                     nbin=15,
                     rcut=10,
                     rotation=None):

                self.m=m
                if rotation==None:
                    self.x=pos[:,0]
                    self.y=pos[:,1]
                    self.z=pos[:,2]
                    self.vx=vel[:,0]
                    self.vy=vel[:,1]
                    self.vz=vel[:,2]
                else:
                    kk,=np.where(pos[:,0]**2+pos[:,1]**2+pos[:,2]**2<rotation)
                    x,y,z,vx,vy,vz=rot.rot1((self.m)[kk],(pos[:,0])[kk],(pos[:,1])[kk],(pos[:,2])[kk],(vel[:,0])[kk],(vel[:,1])[kk],(vel[:,2])[kk])  
                    self.x=x
                    self.y=y
                    self.z=z
                    self.vx=vx
                    self.vy=vy
                    self.vz=vz
                    
                self.nbin=nbin
                self.rcut=rcut
                self.rcil=np.sqrt(self.x**2+self.y**2)
                
                

                 

        def a2max(self):


                A2v=np.ndarray(self.nbin)
                out=np.ndarray([2,self.nbin])
                rbin=bines.rbin1(self.rcil,np.ones(len(self.m)),self.nbin)

                for j in range(0,self.nbin):
                        if j==0:
                                kk,=np.where(self.rcil<=rbin[j])
                        else:
                                kk,=np.where((self.rcil>rbin[j-1]) & (self.rcil<=rbin[j]))

                        xnk=self.x[kk]
                        ynk=self.y[kk]
                        titaj=np.ndarray(np.size(kk))
                        for ii in range(np.size(kk)):
                                titaj[ii]=math.atan2(ynk[ii],xnk[ii])


                        rj=np.sqrt(self.x[kk]**2+self.y[kk]**2) 
                        a0=sum(self.m[kk])
                        a2=sum(self.m[kk]*np.cos(2*titaj))
                        b2=sum(self.m[kk]*np.sin(2*titaj))
                        A2v[j]=np.sqrt(a2**2+b2**2)/a0

                        
                kcut,=np.where(rbin<self.rcut)
                A2max=max(A2v[kcut])
                kA2=np.argmax(A2v[kcut]) 
                rbinmax=(rbin[kcut])[kA2]


                return A2max,rbinmax

        def a2(self):

                A2v=np.ndarray(self.nbin)
                phiv=np.ndarray(self.nbin)
                out=np.ndarray([2,self.nbin])
                rbin=bines.rbin1(self.rcil,np.ones(len(self.m)),self.nbin)


                for j in range(0,self.nbin):
                        if j==0:
                                kk,=np.where(self.rcil<=rbin[j])
                        else:
                                kk,=np.where((self.rcil>rbin[j-1]) & (self.rcil<=rbin[j]))

                        kk,=np.where((self.rcil>rbin[j-1]) & (self.rcil<=rbin[j]))
                        xnk=self.x[kk]
                        ynk=self.y[kk]
                        titaj=np.ndarray(np.size(kk))
                        for ii in range(np.size(kk)):
                                titaj[ii]=math.atan2(ynk[ii],xnk[ii])

                        rj=np.sqrt(self.x[kk]**2+self.y[kk]**2) 
                        a0=sum(self.m[kk])
                        a2=sum(self.m[kk]*np.cos(2*titaj))
                        b2=sum(self.m[kk]*np.sin(2*titaj))
                        A2v[j]=np.sqrt(a2**2+b2**2)/a0
                        phiv[j]=math.atan2(b2,a2)

                        A2max=max(A2v)
                        kA2=np.argmax(A2v) 
                        rbinmax=rbin[kA2]

                        return A2v,phiv,rbin